# Домашнее задание 1: Создание и нормализация базы данных

**Выполнила:** Кузнецова Полина

---

## 1. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

# Загружаю данные
file_path = 'customer_and_transaction(2).xlsx'
df_transactions = pd.read_excel(file_path, sheet_name='transaction')
df_customers = pd.read_excel(file_path, sheet_name='customer')

print(f"Загружено транзакций: {len(df_transactions)}")
print(f"Загружено клиентов: {len(df_customers)}")

## 2. Анализ данных

Смотрю, что есть в таблицах и какие проблемы.

In [ ]:
# Смотрю первые строки транзакций
print("Транзакции:")
df_transactions.head()

In [ ]:
# Смотрю первые строки клиентов
print("Клиенты:")
df_customers.head()

In [ ]:
# Проверяю пропущенные значения
print("Пропущенные значения в транзакциях:")
print(df_transactions.isnull().sum())
print("\nПропущенные значения в клиентах:")
print(df_customers.isnull().sum())

## 3. Нормализация

### Проблема 1: Данные о продуктах дублируются

В таблице транзакций для каждой покупки повторяется вся информация о товаре (бренд, цена и т.д.). Это нарушение 2НФ.

In [ ]:
# Создаю таблицу products
product_columns = ['product_id', 'brand', 'product_line', 'product_class', 
                   'product_size', 'list_price', 'standard_cost']
products = df_transactions[product_columns].drop_duplicates(subset=['product_id'])

print(f"Создана таблица products: {len(products)} товаров")
products.head()

### Проблема 2: Транзитивная зависимость postcode → state, country

Штат и страна зависят от почтового индекса, а не от клиента. Это нарушение 3НФ.

In [ ]:
# Создаю таблицу locations
locations = df_customers[['postcode', 'state', 'country']].drop_duplicates()
locations['location_id'] = range(1, len(locations) + 1)

print(f"Создана таблица locations: {len(locations)} локаций")
locations.head()

### Создание нормализованных таблиц

In [ ]:
# Таблица transactions (без данных о продуктах)
transactions = df_transactions[['transaction_id', 'product_id', 'customer_id', 
                                'transaction_date', 'online_order', 'order_status']]

# Таблица customers (с location_id вместо postcode, state, country)
customers = df_customers.merge(locations[['postcode', 'location_id']], on='postcode')
customers = customers.drop(columns=['postcode', 'state', 'country'])
customers = customers.rename(columns={'DOB': 'date_of_birth'})

print(f"Таблица transactions: {len(transactions)} строк")
print(f"Таблица customers: {len(customers)} строк")

## 4. Сохранение данных в CSV

In [ ]:
# Сохраняю все таблицы в CSV
locations.to_csv('locations.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers_norm.csv', index=False)
transactions.to_csv('transactions_norm.csv', index=False)

print("Все таблицы сохранены!")

## 5. SQL-скрипты

### Создание таблиц

In [ ]:
create_tables_sql = """
-- Создание таблиц

DROP TABLE IF EXISTS transactions CASCADE;
DROP TABLE IF EXISTS customers CASCADE;
DROP TABLE IF EXISTS products CASCADE;
DROP TABLE IF EXISTS locations CASCADE;

CREATE TABLE locations (
    location_id INTEGER PRIMARY KEY,
    postcode INTEGER NOT NULL UNIQUE,
    state VARCHAR(50) NOT NULL,
    country VARCHAR(50) NOT NULL
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    brand VARCHAR(100),
    product_line VARCHAR(50),
    product_class VARCHAR(20),
    product_size VARCHAR(20),
    list_price DECIMAL(10,2) NOT NULL,
    standard_cost DECIMAL(10,2)
);

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(100) NOT NULL,
    last_name VARCHAR(100),
    gender VARCHAR(10) NOT NULL,
    date_of_birth DATE,
    job_title VARCHAR(100),
    job_industry_category VARCHAR(100),
    wealth_segment VARCHAR(50) NOT NULL,
    deceased_indicator CHAR(1) NOT NULL,
    owns_car VARCHAR(3) NOT NULL,
    address VARCHAR(255) NOT NULL,
    location_id INTEGER NOT NULL,
    property_valuation INTEGER NOT NULL,
    FOREIGN KEY (location_id) REFERENCES locations(location_id)
);

CREATE TABLE transactions (
    transaction_id INTEGER PRIMARY KEY,
    product_id INTEGER NOT NULL,
    customer_id INTEGER NOT NULL,
    transaction_date DATE NOT NULL,
    online_order BOOLEAN,
    order_status VARCHAR(20) NOT NULL,
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""

# Сохраняю в файл
with open('create_tables.sql', 'w') as f:
    f.write(create_tables_sql)

print("SQL-скрипт создан!")

## Выводы

В результате работы я:

1. Проанализировала исходные данные
2. Выявила проблемы (дублирование данных о продуктах и транзитивная зависимость)
3. Привела базу данных к 3НФ, создав 4 таблицы:
   - locations (локации)
   - products (товары)
   - customers (клиенты)
   - transactions (транзакции)
4. Сохранила данные в CSV файлы
5. Создала SQL-скрипты для создания таблиц

Теперь база данных нормализована и готова к использованию!